# BAG OF WORDS 


  We convert text to a numerical representation called a feature vector. 
  A feature vector can be as simple as a list of numbers.
  
  The bag-of-words model is one of the feature extraction algorithms for text.

1.The first step in this model is defining the vocabulary

2.The second step is to convert sentences into a frequency vector based on the vocabulary.

In [8]:
#Reading data from document
import pandas as pd
df_pre=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")
df=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")

In [30]:
from sklearn import linear_model

In [31]:
#Adjustments to be done for the data
df['content']=df['content'].str.replace(",", "")

In [32]:
df['content']=df['content'].str.replace("'","")

In [33]:
#creating a list for all content
l=[]
for i in range(0,10000):
    l.append(df['content'].iloc[i])
l

['[lijiang tourism sz002033 stock stock financ invest bank recommend baoy share divid half earn princip group]',
 '[chen ling ding red envelop make money abil grab red envelop fight technolog i grab red envelop cash issu chen ding ling kan hongyan burst happi valentin "s" day fan togeth around red envelop â•® â¯ 3â¯ â•\\xadhttp tcnrzdivjf]',
 '[taobao sucker industri ga fire send children nima blind]',
 '[aspect say know everyth laugh cri]',
 '[over zhang]',
 '[lifetim my favorit main bodi hook three togeth burst happi black tong googl]',
 '[on uniqlo dress room sound insul good bad share know almost]',
 '[so ordinari everyday scene also much longer accumul one day got look pictur i feel burst nostalgia age rememb pain accompani]',
 '[overh red xxiii tanggua children stick grab red envelop win i grab littl _ big red overh wang xiansen issu year goat yet come good luck come red envelop tri luck http tcnrzggwg6]',
 '[there good thing share flash note note todo best chine softwar go downl

In [34]:
#code for bag of words model
import numpy as np
import re

#for building vocabulary
def tokenize_sentences(sentences):
    words = []
    for sentence in sentences:
        w = extract_words(sentence)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

def extract_words(sentence):
    ignore_words = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    words = re.sub("[^\w]", " ",  sentence).split() #nltk.word_tokenize(sentence)
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned    

#function which returns feature vector
def bagofwords(sentence, words):
    sentence_words = extract_words(sentence)
    # frequency word count
    bag = np.zeros(len(words),dtype=int)
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag)

In [35]:
#building the vocabulary for the list created
vocabulary1 = tokenize_sentences(l)

In [36]:
l1 = [x for x in vocabulary1 if not (x.isdigit() or x[1:].isdigit())]

In [37]:
b=pd.DataFrame()

In [38]:
#constructing bag of words
a=[]

for i in range(0,10000):
    #b.append(bagofwords(df['content'].iloc[i], vocabulary1),ignore_index=True)
    a.append(bagofwords(df['content'].iloc[i], vocabulary1))

In [40]:
type(a)

list

In [41]:
bow=np.asarray(a)

In [42]:
df_pol=pd.read_csv("E:\\DMA_PRE\\weibo_polarity.csv")

In [43]:
type(bow)
print(bow.shape)

(10000, 16792)


In [44]:
df_pol.columns

Index(['Unnamed: 0', 'u_id', 'm_id', 'forward_count', 'comment_count',
       'like_count', 'content', 'date', 'time', 'content_media_count',
       'content_#_count', 'content_@_count', 'content_?_count',
       'content_!_count', 'content_length', 'content_emoji_count', 'hour',
       'min', 'sec', 'forward_min', 'forward_max', 'forward_median',
       'forward_mean', 'comment_min', 'comment_max', 'comment_median',
       'comment_mean', 'like_min', 'like_max', 'like_median', 'like_mean',
       'Unnamed: 0.1', 'content_spchar', 'non_emoji_content', 'en_content',
       'Unnamed: 1', 'url_rem', 'contentwurl', 'polarity'],
      dtype='object')

In [1]:
#train_bow=bow5[0:8000]

In [2]:
#pred_bow=bow5[8001:10000]

# Linear Regression Model with BOW appened with other features

In [3]:

bow1=np.insert(bow,16792,df_pol["content_media_count"],axis=1)
bow2=np.insert(bow1,16793,df_pol["forward_median"],axis=1)
bow3=np.insert(bow2,16794,df_pol["comment_median"],axis=1)
bow4=np.insert(bow3,16795,df_pol["like_median"],axis=1)
bow5=np.insert(bow4,16796,df_pol["polarity"],axis=1)

X_train1=train_bow
X_test1=pred_bow
Y_train1=train_df[["forward_count","like_count","comment_count"]]
Y_test1=predict_df[["forward_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

In [ ]:
np.savetxt("E://DMA_PRE//weibo_predict_resultbow.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
result=pd.read_csv("E://DMA_PRE//weibo_predict_resultbow.csv")

In [4]:
train_real_pred = Y_test1
train_real_pred['fp']=result['forward_count'].values
train_real_pred['cp']=result['comment_count'].values
train_real_pred['lp']=result['like_count'].values
train_real_pred=train_real_pred.round()
print ("Score on the training set:{0:.2f}%".format(precision(train_real_pred.values)*100))

In [16]:
t=a[0:8000]

In [17]:
cv=a[8001:10000]

In [18]:
df = pd.DataFrame({'bow':a})

In [19]:
df.to_csv('E:\\DMA_PRE\\bow_df.csv')

In [20]:
df.shape

(10000, 1)

In [21]:
df.iloc[0]

bow    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 0, dtype: object

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [4]:
#from evaluation import precision

In [7]:
df1=pd.read_csv("E:\\DMA_PRE\\PREPROCESSED.csv")
df_bow=pd.read_csv("E:\\DMA_PRE\\bow_df.csv")

In [9]:
result = pd.concat([df1, df], axis=1)

In [13]:
df1

,u_id,m_id,forward_count,comment_count,like_count,date,time,content_media_count,content
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,0,0,0,2015-02-23,17:41:29,0.0,"['lijiang', 'tourism', 'sz002033', 'stock', 's..."
1,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,0,0,0,2015-02-14,12:49:58,1.0,"['chen', 'ling', 'ding', 'red', 'envelop', 'ma..."
2,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,0,0,0,2015-03-31,13:58:06,0.0,"['taobao', 'sucker', 'industri', 'ga', 'fire',..."
3,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,0,4,3,2015-06-11,20:39:57,0.0,"['aspect', 'say', 'know', 'everyth', 'laugh', ..."
4,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,0,0,0,2015-03-10,18:02:38,0.0,"['over', 'zhang']"
5,d80f3d3c5c1d658e82b837a4dd1af849,bfc0819b83ec59ce767287077f2b3507,0,0,0,2015-02-13,01:09:41,1.0,"['lifetim', 'my', 'favorit', 'main', 'bodi', '..."
6,f349a67d1cd7c8683c5bbc5f8486e193,83674a60e5310195fc35d97ea8f45c46,0,0,0,2015-07-15,01:16:24,1.0,"['on', 'uniqlo', 'dress', 'room', 'sound', 'in..."
7,24b621c98f2594b698c0b1d60c9ae6db,2cbd3d514ed5ad3dab81aa043c8b3d0a,0,0,0,2015-05-19,10:24:57,0.0,"['so', 'ordinari', 'everyday', 'scene', 'also'..."
8,e44d81d630e4f382f657e72aa4b685da,8a88a25f9f26ed9f79080eaacc1a8668,0,1,0,2015-02-11,11:03:36,1.0,"['overh', 'red', 'xxiii', 'tanggua', 'children..."
9,fbe6c953632e1b3dda66cf6118b6ab12,f359a74cb4ac6150a3af8325eda04ea0,0,0,0,2015-03-22,00:54:34,1.0,"['there', 'good', 'thing', 'share', 'flash', '..."


In [25]:
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [29]:
result

,u_id,m_id,forward_count,comment_count,like_count,date,time,content_media_count,content,bow
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,0,0,0,2015-02-23,17:41:29,0.0,"['lijiang', 'tourism', 'sz002033', 'stock', 's...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,0,0,0,2015-02-14,12:49:58,1.0,"['chen', 'ling', 'ding', 'red', 'envelop', 'ma...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,0,0,0,2015-03-31,13:58:06,0.0,"['taobao', 'sucker', 'industri', 'ga', 'fire',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,0,4,3,2015-06-11,20:39:57,0.0,"['aspect', 'say', 'know', 'everyth', 'laugh', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,0,0,0,2015-03-10,18:02:38,0.0,"['over', 'zhang']","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,d80f3d3c5c1d658e82b837a4dd1af849,bfc0819b83ec59ce767287077f2b3507,0,0,0,2015-02-13,01:09:41,1.0,"['lifetim', 'my', 'favorit', 'main', 'bodi', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,f349a67d1cd7c8683c5bbc5f8486e193,83674a60e5310195fc35d97ea8f45c46,0,0,0,2015-07-15,01:16:24,1.0,"['on', 'uniqlo', 'dress', 'room', 'sound', 'in...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,24b621c98f2594b698c0b1d60c9ae6db,2cbd3d514ed5ad3dab81aa043c8b3d0a,0,0,0,2015-05-19,10:24:57,0.0,"['so', 'ordinari', 'everyday', 'scene', 'also'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,e44d81d630e4f382f657e72aa4b685da,8a88a25f9f26ed9f79080eaacc1a8668,0,1,0,2015-02-11,11:03:36,1.0,"['overh', 'red', 'xxiii', 'tanggua', 'children...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,fbe6c953632e1b3dda66cf6118b6ab12,f359a74cb4ac6150a3af8325eda04ea0,0,0,0,2015-03-22,00:54:34,1.0,"['there', 'good', 'thing', 'share', 'flash', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# BAG OF WORDS USING COUNTER VECTORIZER


In [30]:
df.to_csv('E:\\DMA_PRE\\pre_bow.csv')

In [51]:
df1=pd.read_csv('E:\\DMA_PRE\\pre_bow.csv')
train_df=df1[0:8000]
train_df.shape

(8000, 2)

In [52]:
predict_df=df1[8001:10000]

In [53]:
train_l=[]
for i in range(0,8000):
    train_l.append(df_pre['content'].iloc[i])
len(train_l)

8000

In [55]:
pred_l=[]
for i in range(8001,10000):
    pred_l.append(df_pre['content'].iloc[i])
len(pred_l)

1999

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
vect=CountVectorizer()

In [58]:
vect.fit(train_l)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [59]:
x=vect.transform(train_l)

In [60]:
#vocabulary=vect.get_feature_names()
#print(vocabulary)

In [61]:
print(x[0,:])

  (0, 1593)	1
  (0, 1608)	1
  (0, 3688)	1
  (0, 3923)	1
  (0, 4535)	1
  (0, 5181)	1
  (0, 5286)	1
  (0, 6171)	1
  (0, 6991)	1
  (0, 9310)	1
  (0, 9783)	1
  (0, 10657)	1
  (0, 11334)	2
  (0, 11648)	1
  (0, 13110)	1


In [62]:
vect = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 100)

In [63]:
print(vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [25]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import precision_score

In [ ]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()
off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_forward = predict_df.forward_count

X_train1=off_train_data_features
X_test1= off_test_data_features
Y_train1=train_df[["forward_count"]]
Y_test1=predict_df[["forward_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train1,Y_train1)
pred1=lm.predict(X_test1)
pred1=pred1.round()
pred1=(np.maximum(pred1,0.))

np.savetxt("E://DMA_PRE//weibo_predict_result1.csv",pred1,delimiter=',',header="forward_count",comments="")
result1=pd.read_csv("E://DMA_PRE//weibo_predict_result1.csv")


In [99]:
print(model.coef_)

[[  0.63778488  -2.02974382  -2.92909528  -3.84533907  -0.21627466
   -2.32174305  -2.45337196  -1.81628588   6.17567497   4.60558578
   -1.07036463  -5.83678995  -2.38168093  -0.70474864   1.59624475
    4.57115665  -5.42310572   1.52719178  -0.12723111  -1.51964219
   -3.29106785   0.78682411  -2.40871863   0.90247276   2.44968886
    0.49228419  -0.86148112   2.28047971   6.76366397   0.06884898
   -1.2165656    0.35548585  16.01798259  -0.51369189  -0.91623708
   -1.18206804  -1.90875443  -1.4912602    1.44890962   6.33928803
    0.15601931   0.4053814   -2.87548232  -0.4854406   -0.6041513
   -0.82895966   4.50853547  -2.49883014   1.27792721  -0.04919261
    1.57614485   1.00368018  -6.1866169   28.3169986   -2.81772194
    1.72140179   0.38122856  -2.51440054  -2.50996279  -1.22834406
   -1.96121174  -2.49587541  -1.93333704  -2.55533174   2.8517133
   -2.0915621   -2.70341256  -2.10523973  -0.23648307   1.41910874
   -1.19494569   0.13469309   0.36080587   4.91712953   6.982392

In [100]:
print(model.intercept_)

[ 0.26692297]


In [101]:
 mean_squared_error(Y_test1,pred1)

1730.0490245122562

In [49]:

off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()
off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_forward = predict_df.forward_count

X_train2=off_train_data_features
X_test2= off_test_data_features
Y_train2=train_df[["like_count"]]
Y_test2=predict_df[["like_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train2,Y_train2)
pred2=lm.predict(X_test2)
pred2=pred2.round()
pred2=(np.maximum(pred2,0.))

np.savetxt("E://DMA_PRE//weibo_predict_result2.csv",pred2,delimiter=',',header="like_count",comments="")
result2=pd.read_csv("E://DMA_PRE//weibo_predict_result2.csv")


In [50]:
from sklearn.metrics import mean_squared_error

In [96]:
 mean_squared_error(Y_test2,pred2)

array([ 1091.00950475])

In [93]:
off_train_data_features = vect.fit_transform(train_l)
off_train_data_features = off_train_data_features.toarray()
off_train_data_forward = train_df.forward_count

off_test_data_features = vect.fit_transform(pred_l)
off_test_data_features = off_test_data_features.toarray()
off_test_data_forward = predict_df.forward_count

X_train3=off_train_data_features
X_test3= off_test_data_features
Y_train3=train_df[["comment_count"]]
Y_test3=predict_df[["comment_count"]]

lm=linear_model.LinearRegression()
model=lm.fit(X_train3,Y_train3)
pred3=lm.predict(X_test3)
pred3=pred3.round()
pred3=(np.maximum(pred3,0.))

np.savetxt("E://DMA_PRE//weibo_predict_result3.csv",pred3,delimiter=',',header="comment_count",comments="")
result3=pd.read_csv("E://DMA_PRE//weibo_predict_result3.csv")


In [94]:
 mean_squared_error(Y_test3,pred3)

51.886443221610804